In [162]:
class MovingAverage():
    def __init__(self, pts):
        pts = int(pts)
        if pts < 1:
            pts = 1
        self._pts = [0]*pts
        self._npts = pts
        self._cur = 0
        self._sum = 0
        self._used = 0
        
    def change_size(self, pts):
        pts = int(pts)
        if pts < 1:
            pts = 1
        
        if pts > self._npts:
            if self._used == self._npts:
                self._pts = calc_new_fullarray_nsmpl_ge(pts, self._pts, self._cur)
                self._cur = self._used
            else:
                self._pts.extend(0 for _ in range(pts - self._npts))
        elif pts < self._npts:
            self._used = min(self._used, pts)
            self._pts = calc_new_fullarray_nsmpl_le(pts, self._pts, self._cur)
            self._cur = 0
            
        self._npts = pts
        self._sum = sum(self._pts)
        
    def add(self, pt):
        cp = self._pts[self._cur]
        self._pts[self._cur] = pt
        self._inc()
        self._sum = self._sum + pt - cp
    
    def _inc(self):
        self._used += 1
        if self._used > self._npts:
            self._used = self._npts
        self._cur += 1
        if self._cur >= self._npts:
            self._cur = 0
            
    def ave(self):
        return self._sum / self._used
    
    def process(self, pt):
        self.add(pt)
        return self.ave()
    
    __call__ = process

In [7]:
ma = MovingAverage(3)
print(ma.process(2))
print(ma.process(3))
print(ma.process(4))
print(ma.process(5))
print(ma.process(6))

2.0
2.5
3.0
4.0
5.0


In [158]:
def calc_new_fullarray_nsmpl_ge(nsmpl, old, idx):
    new = [0] * nsmpl
    ll = len(old)
    for i in range(ll):
        new[i] = old[(i+idx)%ll]
    return new

def calc_new_fullarray_nsmpl_le(nsmpl, old, idx):
    new = [0] * nsmpl
    ll = len(old)
    for i in range(nsmpl):
        new[i] = old[(idx - nsmpl + i)%ll]
    return new

class lvmovingave():
    def __init__(self):
        self.first = True
        
    def next(self, val, nsmpl):
        # sanitize
        if nsmpl < 1:
            nsmpl = 1
        nsmpl = int(nsmpl)
        
        if self.first:
            self.first = False
            self.val_to_ave = nsmpl
            self.array = [0] * nsmpl
            self.sum = val
            self.array[0] = val
            self.i = 0
            self.full_array = False
        else:
            if nsmpl > self.val_to_ave:
                if self.full_array:
                    # tricky, have to copy in correct order
                    new = calc_new_fullarray_nsmpl_ge(nsmpl, self.array, self.i)
                    self.i = len(self.array)
                    self.array = new
                else:
                    # not full yet, easy to increase size. 
                    # self.i and self.array stay the same. 
                    self.array += [0] * (nsmpl - len(self.array))
                    assert len(self.array) == nsmpl
                    
                # size increased, therefore array is not full
                # also, sum does not change
                self.full_array = False
            elif nsmpl < self.val_to_ave:
                # shrink array. 
                
                # if we'll be full after shrinking, mark us as full
                # this is important because self.i gets reset here,
                # before the normal full_array check.
                if self.i + 1 >= nsmpl:
                    self.full_array = True
                self.array = calc_new_fullarray_nsmpl_le(nsmpl, self.array, self.i)
                self.i = 0
                self.sum = sum(self.array)
            
            self.val_to_ave = nsmpl
            self.sum = self.sum - self.array[self.i] + val
            self.array[self.i] = val
            
        n = self.i + 1
        if n < self.val_to_ave:
            self.i = n
            if self.full_array:
                div = self.val_to_ave
            else:
                div = n
        else:
            self.full_array = True
            self.i = 0
            div = self.val_to_ave
        return self.sum / div
    
    __call__ = next

In [159]:
ave = lvmovingave()
res = ave(1,5)
assert res == 1
assert ave.array == [1, 0, 0, 0, 0], ave.array
res = ave(2,5)
assert res == 1.5
assert ave.array == [1, 2, 0, 0, 0], ave.array
res = ave(3,3)
assert res == 2, (res, ave.array, ave.sum)
assert ave.array == [3, 1, 2], ave.array

In [161]:
# test strinking array

ave = lvmovingave()
res = ave(1,5)
assert res == 1
assert ave.array == [1, 0, 0, 0, 0], ave.array
res = ave(2,5)
assert res == 1.5
assert ave.array == [1, 2, 0, 0, 0], ave.array
res = ave(3,5)
assert res == 2
assert ave.array == [1, 2, 3, 0, 0], ave.array
res = ave(4,5)
assert res == 2.5
assert ave.array == [1, 2, 3, 4, 0], ave.array
res = ave(5,3)
assert ave.array == [5, 3, 4], ave.array
assert res == 4, res

ave = lvmovingave()
res = ave(1,5)
assert res == 1
assert ave.array == [1, 0, 0, 0, 0], ave.array
res = ave(2,5)
assert res == 1.5
assert ave.array == [1, 2, 0, 0, 0], ave.array
res = ave(3,5)
assert res == 2
assert ave.array == [1, 2, 3, 0, 0], ave.array
res = ave(4,3)
assert res == 3
assert ave.array == [4, 2, 3], ave.array
res = ave(5,3)
assert ave.array == [4, 5, 3], ave.array
assert res == 4, res

ave = lvmovingave()
res = ave(1,5)
assert res == 1
assert ave.array == [1, 0, 0, 0, 0], ave.array
res = ave(2,5)
assert res == 1.5
assert ave.array == [1, 2, 0, 0, 0], ave.array
res = ave(3,3)
assert res == 2, (res, ave.array, ave.sum)
assert ave.array == [3, 1, 2], ave.array

ave = lvmovingave()
res = ave(1,5)
assert res == 1
assert ave.array == [1, 0, 0, 0, 0], ave.array
res = ave(2,5)
assert res == 1.5
assert ave.array == [1, 2, 0, 0, 0], ave.array
res = ave(3,5)
assert res == 2
assert ave.array == [1, 2, 3, 0, 0], ave.array
res = ave(4,5)
assert res == 2.5
assert ave.array == [1, 2, 3, 4, 0], ave.array
res = ave(5,5)
assert ave.array == [1, 2, 3, 4, 5], ave.array
assert res == 3, res
res = ave(6,3)
assert ave.array == [6, 4, 5], ave.array
assert res == 5, res

ave = lvmovingave()
res = ave(1,5)
assert res == 1
assert ave.array == [1, 0, 0, 0, 0], ave.array
res = ave(2,5)
assert res == 1.5
assert ave.array == [1, 2, 0, 0, 0], ave.array
res = ave(3,5)
assert res == 2
assert ave.array == [1, 2, 3, 0, 0], ave.array
res = ave(4,5)
assert res == 2.5
assert ave.array == [1, 2, 3, 4, 0], ave.array
res = ave(5,5)
assert ave.array == [1, 2, 3, 4, 5], ave.array
assert res == 3, res
res = ave(6,5)
assert ave.array == [6, 2, 3, 4, 5], ave.array
assert res == sum(ave.array)/len(ave.array)
res = ave(7,3)
assert ave.array == [7, 5, 6], ave.array
assert res == 6, res

# neutral test
ave = lvmovingave()
res = ave(1, 3)
assert res == 1
assert ave.array == [1, 0, 0]
res = ave(2, 3)
assert res == 1.5
assert ave.array == [1, 2, 0]
res = ave(3, 3)
assert res == 2
assert ave.array == [1, 2, 3]
res = ave(4, 3)
assert res == 3
assert ave.array == [4, 2, 3]
res = ave(5, 3)
assert res == 4
assert ave.array == [4, 5, 3]

# test growing array
ave = lvmovingave()
res = ave(1, 3)
assert res == 1
assert ave.array == [1, 0, 0]
res = ave(2, 3)
assert res == 1.5
assert ave.array == [1, 2, 0]
res = ave(3, 5)
assert res == 2
assert ave.array == [1, 2, 3, 0, 0]

ave = lvmovingave()
res = ave(1, 3)
assert res == 1
assert ave.array == [1, 0, 0]
res = ave(2, 5)
assert res == 1.5
assert ave.array == [1, 2, 0, 0, 0]

ave = lvmovingave()
res = ave(1, 3)
assert res == 1
assert ave.array == [1, 0, 0]
res = ave(2, 3)
assert res == 1.5
assert ave.array == [1, 2, 0]
res = ave(3, 3)
assert res == 2
assert ave.array == [1, 2, 3]
res = ave(4, 3)
assert res == 3
assert ave.array == [4, 2, 3]
res = ave(5, 5)
assert res == 14/4
assert ave.array == [2, 3, 4, 5, 0]

ave = lvmovingave()
res = ave(1, 3)
assert res == 1
assert ave.array == [1, 0, 0]
res = ave(2, 3)
assert res == 1.5
assert ave.array == [1, 2, 0]
res = ave(3, 3)
assert res == 2
assert ave.array == [1, 2, 3]
res = ave(4, 3)
assert res == 3
assert ave.array == [4, 2, 3]
res = ave(5, 5)
assert res == 14/4
assert ave.array == [2, 3, 4, 5, 0]

ave = lvmovingave()
res = ave(1, 3)
assert res == 1
assert ave.array == [1, 0, 0]
res = ave(2, 3)
assert res == 1.5
assert ave.array == [1, 2, 0]
res = ave(3, 3)
assert res == 2
assert ave.array == [1, 2, 3]
res = ave(4, 3)
assert res == 3
assert ave.array == [4, 2, 3]
res = ave(5, 3)
assert res == 4
assert ave.array == [4, 5, 3]
res = ave(6, 5)
assert res == 18/4
assert ave.array == [3, 4, 5, 6, 0]
print("done")

done


In [175]:
# test strinking array

# this is a shitty wrapper that lets me reuse the same code while only
# changing the init function call before each test block

def new_ma(n):
    m = MovingAverage(n)
    class _ave():
        def __init__(self, m):
            self.m = m
        def __call__(self, v, n2):
            if n2 != n:
                self.m.change_size(n2)
            return self.m(v)
        @property
        def array(self):
            return self.m._pts
    return _ave(m)
    
ave = new_ma(5)
res = ave(1,5)
assert res == 1
assert ave.array == [1, 0, 0, 0, 0], ave.array
res = ave(2,5)
assert res == 1.5
assert ave.array == [1, 2, 0, 0, 0], ave.array
res = ave(3,5)
assert res == 2
assert ave.array == [1, 2, 3, 0, 0], ave.array
res = ave(4,5)
assert res == 2.5
assert ave.array == [1, 2, 3, 4, 0], ave.array
res = ave(5,3)
assert ave.array == [5, 3, 4], ave.array
assert res == 4, res

ave = new_ma(5)
res = ave(1,5)
assert res == 1
assert ave.array == [1, 0, 0, 0, 0], ave.array
res = ave(2,5)
assert res == 1.5
assert ave.array == [1, 2, 0, 0, 0], ave.array
res = ave(3,5)
assert res == 2
assert ave.array == [1, 2, 3, 0, 0], ave.array
res = ave(4,3)
assert res == 3
assert ave.array == [4, 2, 3], ave.array
res = ave(5,3)
assert ave.array == [4, 5, 3], ave.array
assert res == 4, res

ave = new_ma(5)
res = ave(1,5)
assert res == 1
assert ave.array == [1, 0, 0, 0, 0], ave.array
res = ave(2,5)
assert res == 1.5
assert ave.array == [1, 2, 0, 0, 0], ave.array
res = ave(3,3)
assert res == 2, (res, ave.array, ave.sum)
assert ave.array == [3, 1, 2], ave.array

ave = new_ma(5)
res = ave(1,5)
assert res == 1
assert ave.array == [1, 0, 0, 0, 0], ave.array
res = ave(2,5)
assert res == 1.5
assert ave.array == [1, 2, 0, 0, 0], ave.array
res = ave(3,5)
assert res == 2
assert ave.array == [1, 2, 3, 0, 0], ave.array
res = ave(4,5)
assert res == 2.5
assert ave.array == [1, 2, 3, 4, 0], ave.array
res = ave(5,5)
assert ave.array == [1, 2, 3, 4, 5], ave.array
assert res == 3, res
res = ave(6,3)
assert ave.array == [6, 4, 5], ave.array
assert res == 5, res

ave = new_ma(5)
res = ave(1,5)
assert res == 1
assert ave.array == [1, 0, 0, 0, 0], ave.array
res = ave(2,5)
assert res == 1.5
assert ave.array == [1, 2, 0, 0, 0], ave.array
res = ave(3,5)
assert res == 2
assert ave.array == [1, 2, 3, 0, 0], ave.array
res = ave(4,5)
assert res == 2.5
assert ave.array == [1, 2, 3, 4, 0], ave.array
res = ave(5,5)
assert ave.array == [1, 2, 3, 4, 5], ave.array
assert res == 3, res
res = ave(6,5)
assert ave.array == [6, 2, 3, 4, 5], ave.array
assert res == sum(ave.array)/len(ave.array)
res = ave(7,3)
assert ave.array == [7, 5, 6], ave.array
assert res == 6, res

# neutral test
ave = new_ma(3)
res = ave(1, 3)
assert res == 1
assert ave.array == [1, 0, 0]
res = ave(2, 3)
assert res == 1.5
assert ave.array == [1, 2, 0]
res = ave(3, 3)
assert res == 2
assert ave.array == [1, 2, 3]
res = ave(4, 3)
assert res == 3
assert ave.array == [4, 2, 3]
res = ave(5, 3)
assert res == 4
assert ave.array == [4, 5, 3]

# test growing array
ave = ave = new_ma(3)
res = ave(1, 3)
assert res == 1
assert ave.array == [1, 0, 0]
res = ave(2, 3)
assert res == 1.5
assert ave.array == [1, 2, 0]
res = ave(3, 5)
assert res == 2
assert ave.array == [1, 2, 3, 0, 0]

ave = ave = new_ma(3)
res = ave(1, 3)
assert res == 1
assert ave.array == [1, 0, 0]
res = ave(2, 5)
assert res == 1.5
assert ave.array == [1, 2, 0, 0, 0]

ave = ave = new_ma(3)
res = ave(1, 3)
assert res == 1
assert ave.array == [1, 0, 0]
res = ave(2, 3)
assert res == 1.5
assert ave.array == [1, 2, 0]
res = ave(3, 3)
assert res == 2
assert ave.array == [1, 2, 3]
res = ave(4, 3)
assert res == 3
assert ave.array == [4, 2, 3]
res = ave(5, 5)
assert res == 14/4
assert ave.array == [2, 3, 4, 5, 0]

ave = ave = new_ma(3)
res = ave(1, 3)
assert res == 1
assert ave.array == [1, 0, 0]
res = ave(2, 3)
assert res == 1.5
assert ave.array == [1, 2, 0]
res = ave(3, 3)
assert res == 2
assert ave.array == [1, 2, 3]
res = ave(4, 3)
assert res == 3
assert ave.array == [4, 2, 3]
res = ave(5, 5)
assert res == 14/4
assert ave.array == [2, 3, 4, 5, 0]

ave = ave = new_ma(3)
res = ave(1, 3)
assert res == 1
assert ave.array == [1, 0, 0]
res = ave(2, 3)
assert res == 1.5
assert ave.array == [1, 2, 0]
res = ave(3, 3)
assert res == 2
assert ave.array == [1, 2, 3]
res = ave(4, 3)
assert res == 3
assert ave.array == [4, 2, 3]
res = ave(5, 3)
assert res == 4
assert ave.array == [4, 5, 3]
res = ave(6, 5)
assert res == 18/4
assert ave.array == [3, 4, 5, 6, 0]
print("done")

done


In [ ]:
# cal_new test 
arr = [4, 2, 3]
idx = 1
ssum = sum(arr)
arr = calc_new_fullarray_nsmpl_ge(5, arr, idx)
idx = 3
ssum = ssum - arr[idx] + 5
arr[idx] = 5
arr, ssum, ssum == sum(arr), ssum / (idx + 1), sum(arr) / 4

In [33]:
arr = [5, 6, 2, 3, 4]
exp = [7,5,6]
idx = 2
ssum = sum(arr)
arr = calc_new_fullarray_nsmpl_le(3, arr, idx)
idx = 0
val = 7
ssum = sum(arr) - arr[idx] + val
arr[idx] = val
arr, ssum, ssum == sum(arr)

([7, 5, 6], 18, True)

In [42]:
ave = lvmovingave()
m = MovingAverage(3)
for i in range(10):
    print(ave(i, 3), m(i))

0.0 0.0
0.5 0.5
1.0 1.0
2.0 2.0
3.0 3.0
4.0 4.0
5.0 5.0
6.0 6.0
7.0 7.0
8.0 8.0


In [58]:
ave.array

[5, 3, 1]

In [59]:
ave.sum

9

In [55]:
ave.i

1